# 0. Importando Pandas

In [47]:
import pandas as pd

# 1. Importando dados 

In [48]:
path = 'us-shein-automotive-4110.csv'
auto = pd.read_csv(path, sep=None, engine='python')

In [49]:
# Defini o caminho do arquivo CSV como uma variável e
# usei o método read_csv com detecção automática
# de delimitador e motor Python

# 2. Analisando a estrutura dos dados

In [50]:
auto.head()

,goods-title-link--jump,goods-title-link--jump href,price,discount,selling_proposition,goods-title-link
0,"1pc/2PCS Car Cup Coaster, Bling Cup Holder Ins...",https://us.shein.com/1pc-2PCS-Car-Cup-Coaster-...,$0.68,-60%,NaN,NaN
1,Jesus The Way Truth Life Sticker For Car Windo...,https://us.shein.com/Jesus-The-Way-Truth-Life-...,$1.60,-20%,500+ sold recently,NaN
2,2PC Car Rearview Mirror Rain Eyebrow And Rain ...,https://us.shein.com/2PC-Car-Rearview-Mirror-R...,$1.60,NaN,200+ sold recently,NaN
3,"2pcs Car Rearview Mirror Rainproof Film, High-...",https://us.shein.com/2pcs-Car-Rearview-Mirror-...,$0.93,-34%,200+ sold recently,NaN
4,2pcs Multifunctional Car Hook,https://us.shein.com/2pcs-Multifunctional-Car-...,$1.12,-20%,1.9k+ sold recently,NaN


In [51]:
auto.tail()

,goods-title-link--jump,goods-title-link--jump href,price,discount,selling_proposition,goods-title-link
4104,NaN,NaN,$18.50,NaN,NaN,2 Pcs 3 Lens 4 Lens Super Bright Light Motorcy...
4105,NaN,NaN,$21.00,NaN,NaN,680 Car Fastener Clips Auto Body Push Retainer...
4106,NaN,NaN,$20.33,-25%,NaN,"Cargo Liner For SUV Dog, Rear Seat Hanging Car..."
4107,NaN,NaN,$17.40,NaN,40+ sold recently,Motorcycle Cover Universal Waterproof Accessor...
4108,NaN,NaN,$59.99,NaN,NaN,Unbeatablesale 16 In. Wheel Covers For Camry


In [52]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4109 entries, 0 to 4108
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   goods-title-link--jump       40 non-null     object
 1   goods-title-link--jump href  40 non-null     object
 2   price                        4109 non-null   object
 3   discount                     1373 non-null   object
 4   selling_proposition          2874 non-null   object
 5   goods-title-link             4069 non-null   object
dtypes: object(6)
memory usage: 192.7+ KB


In [53]:
# Verifica-se que há valores nulos, a nomeação das colunas
# não está clara, os datatypes precisam ser trabalhados para
# facilitar futuras análises, ao usar 'head' e 'tail' também vimos
# que há duas colunas com valores relativos ao nome do produto

# 3. Limpeza dos dados

In [54]:
auto = auto.rename(
    {
        'goods-title-link--jump': 'product',
        'price': 'price_USD',
        'goods-title-link--jump href': 'product_link'
    },
    axis=1
)

In [55]:
# Renomeia as colunas

In [56]:
auto['product_name'] = (
    auto['product']
    .fillna(auto['goods-title-link'])
)

In [57]:
# Cria a coluna 'product_name' preenchendo os valores
# nulos da coluna 'product' com os valores da coluna
# 'goods-title-link'

In [58]:
auto = auto.drop(
    columns=['product', 'goods-title-link']
)

In [59]:
# Após criada a coluna product_name podemos remover
# as colunas 'product' e 'goods-title-link'

In [60]:
auto = auto[[
    'product_name',
    'price_USD',
    'selling_proposition',
    'discount',
    'product_link'
]]

In [61]:
# Ordenando as colunas atualizando a variável 'auto'

In [62]:
auto['price_USD'] = (
    auto['price_USD']
    .str.replace('$', '', regex=False)
)

In [63]:
# Removendo o $ pra em seguida transformar em float 

In [64]:
auto['price_USD'] = (
    auto['price_USD']
    .astype(float)
)

In [65]:
# optei por transformar em float pra facilitar análises
# quantitativas e me certifiquei de explicitar a moeda
# no rótulo da coluna

In [66]:
auto['discount'] = auto['discount'].str.rstrip('%').astype(float) / 100.0

In [67]:
# Optei por transformar a coluna discount em float 
# e converter pra decimal pra facilitar operações matemáticas

In [68]:
auto['discount'] = (
    auto['discount']
    .fillna(0)
)

In [69]:
# Preenchendo os valores nulos da coluna discount com 0

In [70]:
auto.loc[:, 'product_link'] = (
    auto['product_link']
    .fillna('Missing')
)

In [71]:
# Preenchendo os valores nulos da coluna product_link
# como 'Missing'

In [72]:
auto = (
    auto.drop_duplicates()
)

In [73]:
# Removendo duplicatas

In [74]:
auto = auto.reset_index(drop=True)

In [75]:
# reseta o index após remover as duplicatas

In [76]:
auto['is_valid_link'] = (
    auto['product_link'].str.startswith('https://us.shein.com/')
)

In [77]:
# Verifica a integridade dos links pelo domínio
# criando uma nova coluna com valores booleanos

In [78]:
counts = auto['is_valid_link'].value_counts()

print(counts)

is_valid_link
False    3890
True       40
Name: count, dtype: int64


In [79]:
# Agora sabemos quantos links íntegros temos

In [80]:
auto['selling_proposition'] = auto['selling_proposition'].str.replace('k', '00')


In [ ]:
# A coluna 'selling_proposition' possuía valores como '1.9k'
# dessa forma esses valores serão transformados em '1900'

In [81]:
auto['selling_proposition'] = auto['selling_proposition'].fillna('unknown')

In [ ]:
# Assim preenchemos os valores nulos com a string 'unknown'

In [83]:
auto = auto.rename(
    columns={'selling_proposition': 'sold_recently',
            'discount': 'discount_as_percentage'}  
)

In [ ]:
# Renomeei mais uma vez as colunas para que descrevam
# melhor os dados levando em conta as alterações feitas

In [85]:
auto['sold_recently'] = auto['sold_recently'].str.replace(' sold recently', '')


In [89]:
# Remove o trecho ' sold recently' da coluna 'sold_recently'

# 4. Resultado

In [88]:
display(auto)

,product_name,price_USD,sold_recently,discount_as_percentage,product_link,is_valid_link
0,"1pc/2PCS Car Cup Coaster, Bling Cup Holder Ins...",0.68,unknown,-0.60,https://us.shein.com/1pc-2PCS-Car-Cup-Coaster-...,True
1,Jesus The Way Truth Life Sticker For Car Windo...,1.60,500+,-0.20,https://us.shein.com/Jesus-The-Way-Truth-Life-...,True
2,2PC Car Rearview Mirror Rain Eyebrow And Rain ...,1.60,200+,0.00,https://us.shein.com/2PC-Car-Rearview-Mirror-R...,True
3,"2pcs Car Rearview Mirror Rainproof Film, High-...",0.93,200+,-0.34,https://us.shein.com/2pcs-Car-Rearview-Mirror-...,True
4,2pcs Multifunctional Car Hook,1.12,1.900+,-0.20,https://us.shein.com/2pcs-Multifunctional-Car-...,True
...,...,...,...,...,...,...
3925,Unbeatablesale 8 Lbs Smoke Hickory Wood Blocks,24.58,unknown,0.00,Missing,False
3926,VIVOHOME 1100 LBS Motorcycle Lift Scissor Jack...,72.99,unknown,-0.72,Missing,False
3927,Unbeatablesale Arctic Warrior Tip Up,33.75,unknown,0.00,Missing,False
3928,Unbeatablesale 17972 4.5 Gal Oval Drain Pan,38.87,unknown,0.00,Missing,False
